# Notebook 06: Full Inference for Submission
**Mục tiêu:**
1. Chạy pipeline dự đoán trên **TẤT CẢ** các tập dữ liệu: Train, Validation, và Test.
2. Sinh ra file `pred.json` cho từng bài báo.
3. Sắp xếp kết quả vào đúng cấu trúc thư mục để nộp bài (Submission Ready).

**Model:** Decision Tree/XGBoost với **6 selected features**

**Cấu trúc Output:**
```bash
submission_final/
├── <Student_ID>/
│   ├── <paper_id_1>/
│   │   └── pred.json
│   ├── <paper_id_2>/
│   │   └── pred.json
...
```

In [1]:
import pandas as pd
import numpy as np
import json
import os
import sys
import joblib
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Setup path để import src module
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

# Import từ src.ml module (Sử dụng lại các hàm đã refactor)
from src.ml import (
    load_json,
    save_json,
    normalize_text_basic,
    compute_pairwise_features,
    compute_tfidf_cosine_single,
    parse_bibtex_smart,
    transform_to_paper_based
)

pd.set_option('display.max_columns', None)
print("✅ Libraries và src.ml modules imported successfully!")

# --- DANH SÁCH 6 FEATURES ĐÃ CHỌN (phải khớp với Notebook 03/04) ---
SELECTED_FEATURES = [
    'feat_title_tfidf_cosine',
    'feat_title_len_diff',
    'feat_auth_jaccard',
    'feat_year_match',
    'feat_id_match',
    'feat_first_auth_match',
]
print(f"📝 Sử dụng {len(SELECTED_FEATURES)} selected features")

✅ Libraries và src.ml modules imported successfully!
📝 Sử dụng 6 selected features


In [2]:
# --- CẤU HÌNH QUAN TRỌNG ---

# 1. Thông tin sinh viên (Để tạo folder nộp bài)
STUDENT_ID = "23127011" 

# 2. Đường dẫn dữ liệu
DATASET_ROOT = '../../dataset_final'  # Chứa labels.json cho groundtruth
DATA_FOLDER = '../../data_output_v2'  # Chứa refs.bib + references.json đầy đủ
PARTITIONS = ['train', 'validation', 'test']

# 3. Đường dẫn Model
MODEL_PATH = '../../dataset_final/models/best_matcher.pkl'
FEATURE_NAME_PATH = '../../dataset_final/models/feature_names.pkl'

# 4. Thư mục Output
SUBMISSION_DIR = f'submission_final/{STUDENT_ID}'

if not os.path.exists(SUBMISSION_DIR):
    os.makedirs(SUBMISSION_DIR)
    print(f"📁 Created submission directory: {os.path.abspath(SUBMISSION_DIR)}")
else:
    print(f"📁 Saving to existing directory: {os.path.abspath(SUBMISSION_DIR)}")

print(f"📂 Dataset (labels): {os.path.abspath(DATASET_ROOT)}")
print(f"📂 Data (refs+references): {os.path.abspath(DATA_FOLDER)}")


📁 Created submission directory: d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\23127011\notebooks\submission_final\23127011
📂 Dataset (labels): d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\dataset_final
📂 Data (refs+references): d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\data_output_v2


In [3]:
# --- OPTIMIZED HELPER FUNCTIONS WITH TIMING ---
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import time

# Global timing stats
TIMING_STATS = {
    'feature_compute': 0.0,
    'tfidf_compute': 0.0,
    'model_predict': 0.0,
    'total_queries': 0
}

def batch_compute_features(pairs_list):
    """Compute features cho batch pairs - VECTORIZED."""
    if not pairs_list:
        return []
    
    results = []
    for row in pairs_list:
        feats = compute_pairwise_features(row)
        results.append(feats)
    return results


def rank_paper_fast(queries, candidates_list, model, feature_names):
    """
    Ranking tất cả queries của 1 paper cùng lúc - SUPER OPTIMIZED.
    
    KEY OPTIMIZATION: 
    - Pre-compute TF-IDF vectors cho candidates 1 LẦN
    - Reuse cho tất cả queries
    """
    global TIMING_STATS
    
    if not candidates_list:
        return {}
    
    predictions = {}
    
    # Pre-normalize titles 1 lần
    cand_titles = [normalize_text_basic(c.get('cand_title', '')) for c in candidates_list]
    cand_ids = [c['cand_id'] for c in candidates_list]
    query_titles = [normalize_text_basic(q.get('bib_title', '')) for q in queries]
    
    # === PRE-COMPUTE TF-IDF CHO TOÀN BỘ PAPER (1 LẦN DUY NHẤT) ===
    t_tfidf_start = time.time()
    
    # Gộp tất cả unique titles (queries + candidates)
    all_titles = list(set(query_titles + cand_titles))
    
    try:
        # Fit vectorizer 1 lần duy nhất cho paper này
        vectorizer = TfidfVectorizer(analyzer='char_wb', ngram_range=(2, 4), min_df=1)
        vectorizer.fit(all_titles)
        
        # Transform candidates 1 lần
        cand_vecs = vectorizer.transform(cand_titles)
        
        # Transform queries 1 lần  
        query_vecs = vectorizer.transform(query_titles)
        
        # Pre-compute tất cả TF-IDF scores (queries x candidates)
        # Shape: (n_queries, n_candidates)
        tfidf_matrix = cosine_similarity(query_vecs, cand_vecs)
        
    except Exception as e:
        # Fallback nếu lỗi
        tfidf_matrix = np.zeros((len(queries), len(candidates_list)))
    
    TIMING_STATS['tfidf_compute'] += time.time() - t_tfidf_start
    
    # === PROCESS TỪNG QUERY (đã có TF-IDF sẵn) ===
    for q_idx, query in enumerate(queries):
        bib_key = query['key']
        
        # Tạo pairs
        pairs = []
        for cand in candidates_list:
            row = {}
            row.update(query)
            row.update(cand)
            pairs.append(row)
        
        # --- PHASE 1: Feature Compute (KHÔNG có TF-IDF) ---
        t0 = time.time()
        feats_list = batch_compute_features(pairs)
        TIMING_STATS['feature_compute'] += time.time() - t0
        
        # Add TF-IDF từ pre-computed matrix
        for c_idx, feats in enumerate(feats_list):
            feats['feat_title_tfidf_cosine'] = tfidf_matrix[q_idx, c_idx]
        
        # Create DataFrame và predict
        df_feats = pd.DataFrame(feats_list)
        
        # Fill missing cols
        for col in feature_names:
            if col not in df_feats.columns:
                df_feats[col] = 0.0
        
        X_input = df_feats[feature_names].values
        
        # --- PHASE 2: Model Predict ---
        t2 = time.time()
        if model:
            scores = model.predict_proba(X_input)[:, 1]
        else:
            scores = np.random.rand(len(pairs))
        TIMING_STATS['model_predict'] += time.time() - t2
        
        # Ranking
        ranked_idx = np.argsort(scores)[::-1][:5]
        top_5 = [cand_ids[i] for i in ranked_idx]
        
        predictions[bib_key] = top_5
        TIMING_STATS['total_queries'] += 1
    
    return predictions


def print_timing_stats():
    """In thống kê thời gian của từng phase."""
    total = TIMING_STATS['feature_compute'] + TIMING_STATS['tfidf_compute'] + TIMING_STATS['model_predict']
    n_queries = max(TIMING_STATS['total_queries'], 1)
    
    print("\n" + "="*50)
    print("⏱️  TIMING BREAKDOWN")
    print("="*50)
    print(f"{'Phase':<25} | {'Total (s)':<12} | {'Avg/Query (ms)':<15} | {'%':<8}")
    print("-"*65)
    
    for phase, label in [
        ('feature_compute', 'Feature Extraction'),
        ('tfidf_compute', 'TF-IDF Cosine'),
        ('model_predict', 'Model Prediction')
    ]:
        t = TIMING_STATS[phase]
        pct = (t / total * 100) if total > 0 else 0
        avg_ms = (t / n_queries) * 1000
        print(f"{label:<25} | {t:>10.2f}s | {avg_ms:>12.2f}ms | {pct:>6.1f}%")
    
    print("-"*65)
    print(f"{'TOTAL':<25} | {total:>10.2f}s | {(total/n_queries)*1000:>12.2f}ms | 100.0%")
    print(f"\n📊 Processed {n_queries} queries")
    print("="*50)


def reset_timing_stats():
    """Reset timing stats."""
    global TIMING_STATS
    TIMING_STATS = {
        'feature_compute': 0.0,
        'tfidf_compute': 0.0,
        'model_predict': 0.0,
        'total_queries': 0
    }


print("✅ SUPER Optimized functions ready (TF-IDF pre-computed per paper).")

✅ SUPER Optimized functions ready (TF-IDF pre-computed per paper).


In [4]:
# --- NEW HELPER FUNCTIONS: Load refs.bib + references.json ---
import bibtexparser

def load_queries_from_bib(bib_path):
    """
    Đọc refs.bib và trả về list queries (TARGET entries cần match).
    
    Returns:
        List[dict]: Mỗi query có: key, bib_title, bib_authors, bib_id, bib_year
    """
    if not os.path.exists(bib_path):
        return []
    
    try:
        with open(bib_path, 'r', encoding='utf-8') as f:
            bib_content = f.read()
        
        parser = bibtexparser.bparser.BibTexParser(common_strings=True)
        parser.ignore_nonstandard_types = True
        bib_db = bibtexparser.loads(bib_content, parser=parser)
        
        queries = []
        for entry in bib_db.entries:
            # Lấy raw text để parse
            raw_text = entry.get('ENTRYTYPE', '') + '{'
            for k, v in entry.items():
                if k not in ['ENTRYTYPE', 'ID']:
                    raw_text += f'{k}={{{v}}}, '
            raw_text += '}'
            
            # Parse bằng hàm smart parser
            parsed = parse_bibtex_smart(raw_text)
            
            queries.append({
                'key': entry.get('ID', ''),
                'bib_title': parsed.get('title', ''),
                'bib_authors': parsed.get('authors', []),
                'bib_id': parsed.get('extracted_id', ''),
                'bib_year': parsed.get('year', '')
            })
        
        return queries
    except Exception as e:
        print(f"⚠️ Error parsing {bib_path}: {e}")
        return []


def load_candidates_from_references(ref_json_path):
    """
    Đọc references.json và trả về list candidates (CANDIDATES pool từ arXiv API).
    
    Returns:
        List[dict]: Mỗi candidate có: cand_id, cand_title, cand_authors, cand_year
    """
    if not os.path.exists(ref_json_path):
        return []
    
    try:
        ref_data = load_json(ref_json_path)
        candidates = []
        
        for arxiv_id, meta in ref_data.items():
            authors = meta.get('authors', [])
            date = meta.get('submission_date', '')
            year = str(date)[:4] if date and len(str(date)) >= 4 else ''
            
            candidates.append({
                'cand_id': arxiv_id,
                'cand_title': meta.get('title', '').lower().strip(),
                'cand_authors': [str(a).lower().strip() for a in authors],
                'cand_year': year
            })
        
        return candidates
    except Exception as e:
        print(f"⚠️ Error loading {ref_json_path}: {e}")
        return []


def extract_groundtruth_from_labels(labels_data):
    """
    Trích xuất groundtruth từ labels.json.
    
    Returns:
        dict: {paper_id: {bib_key: true_arxiv_id}}
    """
    from collections import defaultdict
    gt_map = defaultdict(dict)
    
    for item in labels_data:
        paper_id = item.get('source_paper_id')
        key = item.get('key')
        true_id = item.get('ground_truth', {}).get('id')
        
        if paper_id and key and true_id:
            gt_map[paper_id][key] = true_id
    
    return dict(gt_map)


print("✅ Helper functions for refs.bib + references.json loaded!")


✅ Helper functions for refs.bib + references.json loaded!


In [5]:
# --- LOAD MODEL ---
try:
    model = joblib.load(MODEL_PATH)
    feature_names = joblib.load(FEATURE_NAME_PATH)
    print(f"✅ Model loaded: {type(model).__name__}")
    print(f"📝 Expected features ({len(feature_names)}): {feature_names}")
    
    # Verify features match SELECTED_FEATURES
    if set(feature_names) == set(SELECTED_FEATURES):
        print("✅ Features khớp với SELECTED_FEATURES")
    else:
        print("⚠️ WARNING: Features không khớp! Sử dụng SELECTED_FEATURES")
        feature_names = SELECTED_FEATURES
        
except FileNotFoundError:
    print("❌ CRITICAL ERROR: Model file not found. Cannot proceed.")
    model = None
    feature_names = SELECTED_FEATURES

✅ Model loaded: Pipeline
📝 Expected features (6): ['feat_title_tfidf_cosine', 'feat_title_len_diff', 'feat_auth_jaccard', 'feat_year_match', 'feat_id_match', 'feat_first_auth_match']
✅ Features khớp với SELECTED_FEATURES



## Main Inference Loop
Vòng lặp này sẽ:
1. Duyệt qua từng Partition (Train -> Valid -> Test).
2. Load `labels.json` của partition đó.
3. Transform sang cấu trúc Paper-based.
4. Chạy Ranking Model.
5. Lưu file `pred.json` vào folder tương ứng với Paper ID.


## 🔍 Test & Validation
Kiểm tra logic trước khi chạy full inference:
- Load sample papers từ labels.json
- Đọc refs.bib và references.json
- Hiển thị pairs mẫu

In [6]:
# TEST: Load sample data từ partition đầu tiên
print("🔍 TESTING DATA LOADING...\n")

test_partition = 'train'
label_file = os.path.join(DATASET_ROOT, test_partition, 'labels.json')

if os.path.exists(label_file):
    # 1. Load labels
    labels_data = load_json(label_file)
    print(f"✅ Loaded {len(labels_data)} entries from labels.json")
    
    # 2. Extract groundtruth
    groundtruth_map = extract_groundtruth_from_labels(labels_data)
    paper_ids = list(groundtruth_map.keys())
    
    print(f"✅ Extracted {len(paper_ids)} unique papers")
    print(f"   Sample paper IDs: {paper_ids[:3]}\n")
    
    # 3. Test với paper đầu tiên
    test_paper_id = paper_ids[0]
    print(f"📄 Testing with paper: {test_paper_id}")
    print(f"   Groundtruth entries: {len(groundtruth_map[test_paper_id])}")
    print(f"   Sample groundtruth: {dict(list(groundtruth_map[test_paper_id].items())[:3])}\n")
    
    # 4. Load refs.bib
    paper_dir = os.path.join(DATA_FOLDER, test_paper_id)
    bib_path = os.path.join(paper_dir, 'refs.bib')
    
    if os.path.exists(bib_path):
        queries = load_queries_from_bib(bib_path)
        print(f"✅ Loaded {len(queries)} queries from refs.bib")
        
        if queries:
            print(f"\n📋 Sample Query (TARGET entry cần match):")
            sample_q = queries[0]
            print(f"   Key: {sample_q['key']}")
            print(f"   Title: {sample_q['bib_title'][:100]}...")
            print(f"   Authors: {sample_q['bib_authors'][:3]}")
            print(f"   Year: {sample_q['bib_year']}")
            print(f"   ID: {sample_q['bib_id']}")
    else:
        print(f"⚠️ refs.bib not found at {bib_path}")
    
    # 5. Load references.json
    ref_json_path = os.path.join(paper_dir, 'references.json')
    
    if os.path.exists(ref_json_path):
        candidates = load_candidates_from_references(ref_json_path)
        print(f"\n✅ Loaded {len(candidates)} candidates from references.json")
        
        if candidates:
            print(f"\n🎯 Sample Candidate (từ arXiv API):")
            sample_c = candidates[0]
            print(f"   arXiv ID: {sample_c['cand_id']}")
            print(f"   Title: {sample_c['cand_title'][:100]}...")
            print(f"   Authors: {sample_c['cand_authors'][:3]}")
            print(f"   Year: {sample_c['cand_year']}")
    else:
        print(f"⚠️ references.json not found at {ref_json_path}")
    
    # 6. Show pairing example
    if queries and candidates:
        print(f"\n" + "="*60)
        print(f"📊 PAIRING STATISTICS")
        print(f"="*60)
        print(f"Total queries (refs.bib): {len(queries)}")
        print(f"Total candidates (references.json): {len(candidates)}")
        print(f"Total pairs to rank: {len(queries)} × {len(candidates)} = {len(queries) * len(candidates)}")
        print(f"\nTop-5 predictions per query → {len(queries)} × 5 = {len(queries) * 5} predictions")
        print("="*60)
        
        # Show matching example
        print(f"\n🔗 Example Pairing:")
        print(f"Query Key: {queries[0]['key']}")
        print(f"   → Groundtruth: {groundtruth_map[test_paper_id].get(queries[0]['key'], 'N/A')}")
        print(f"   → Will rank against {len(candidates)} candidates")
        print(f"   → Return top-5 arXiv IDs")
        
else:
    print(f"❌ Label file not found: {label_file}")

print("\n✅ Test completed! Ready to run full inference.")


🔍 TESTING DATA LOADING...

✅ Loaded 13348 entries from labels.json
✅ Extracted 604 unique papers
   Sample paper IDs: ['2403-04225', '2403-03951', '2403-01980']

📄 Testing with paper: 2403-04225
   Groundtruth entries: 48
   Sample groundtruth: {'ref_0': '2305-16213', 'ref_1': '2209-14988', 'ref_2': '2211-10440'}

✅ Loaded 54 queries from refs.bib

📋 Sample Query (TARGET entry cần match):
   Key: ref_0
   Title: Prolificdreamer: High-fidelity and diverse text-to-3d generation with variational score distillation...
   Authors: ['Wang, Zhengyi', 'Lu, Cheng', 'Wang, Yikai']
   Year: 2024
   ID: 

✅ Loaded 48 candidates from references.json

🎯 Sample Candidate (từ arXiv API):
   arXiv ID: 2308-10490
   Title: texture generation on 3d meshes with point-uv diffusion...
   Authors: ['xin yu', 'peng dai', 'wenbo li']
   Year: 

📊 PAIRING STATISTICS
Total queries (refs.bib): 54
Total candidates (references.json): 48
Total pairs to rank: 54 × 48 = 2592

Top-5 predictions per query → 54 × 5 = 270

In [7]:
# Biến để thống kê tổng kết
summary_stats = {
    'train': {'papers': 0, 'mrr': 0, 'queries': 0},
    'validation': {'papers': 0, 'mrr': 0, 'queries': 0},
    'test': {'papers': 0, 'mrr': 0, 'queries': 0}
}

# Reset timing trước khi chạy
reset_timing_stats()

print("🚀 STARTING NEW INFERENCE PIPELINE (refs.bib + references.json)...\n")

for partition in PARTITIONS:
    print(f"🔵 Processing Partition: [{partition.upper()}]")
    partition_start = time.time()
    
    # 1. Load labels.json để lấy groundtruth và paper_ids
    t_load = time.time()
    label_file = os.path.join(DATASET_ROOT, partition, 'labels.json')
    if not os.path.exists(label_file):
        print(f"   ⚠️ Warning: File {label_file} not found. Skipping.")
        continue
        
    labels_data = load_json(label_file)
    if not labels_data:
        print("   ⚠️ Empty labels. Skipping.")
        continue
    print(f"   ⏱️ Load labels: {time.time() - t_load:.2f}s")

    # 2. Extract groundtruth và paper_ids
    t_transform = time.time()
    groundtruth_map = extract_groundtruth_from_labels(labels_data)
    paper_ids = list(groundtruth_map.keys())
    
    summary_stats[partition]['papers'] = len(paper_ids)
    print(f"   ⏱️ Extract groundtruth: {time.time() - t_transform:.2f}s")
    print(f"   ↳ Found {len(paper_ids)} papers. Processing...")
    
    # 3. NEW: Loop qua từng paper và load refs.bib + references.json
    part_mrr_sum = 0
    part_query_count = 0
    
    for paper_id in tqdm(paper_ids, desc=f"   Ranking {partition}"):
        # Xác định đường dẫn paper folder trong data_output_v2
        paper_dir = os.path.join(DATA_FOLDER, paper_id)
        
        if not os.path.exists(paper_dir):
            print(f"   ⚠️ Paper folder not found: {paper_id}")
            continue
        
        # 4. Load QUERIES từ refs.bib (TARGET entries)
        bib_path = os.path.join(paper_dir, 'refs.bib')
        queries = load_queries_from_bib(bib_path)
        
        if not queries:
            print(f"   ⚠️ No queries in {paper_id}/refs.bib")
            continue
        
        # 5. Load CANDIDATES từ references.json (Ground truth pool)
        ref_json_path = os.path.join(paper_dir, 'references.json')
        candidates = load_candidates_from_references(ref_json_path)
        
        if not candidates:
            print(f"   ⚠️ No candidates in {paper_id}/references.json")
            continue
        
        # 6. Run ranking
        predictions = rank_paper_fast(queries, candidates, model, feature_names)
        
        # 7. Build pred.json output
        groundtruth = groundtruth_map.get(paper_id, {})
        
        pred_output = {
            "partition": partition,
            "groundtruth": groundtruth,
            "prediction": predictions
        }
        
        # 8. Calculate MRR
        for query in queries:
            bib_key = query['key']
            true_id = groundtruth.get(bib_key)
            
            if not true_id:
                continue
            
            top_5 = predictions.get(bib_key, [])
            
            if true_id in top_5:
                rank = top_5.index(true_id) + 1
                part_mrr_sum += 1.0 / rank
            
            part_query_count += 1
        
        # 9. Save pred.json
        safe_paper_id = str(paper_id).strip()
        output_dir = os.path.join(SUBMISSION_DIR, safe_paper_id)
        os.makedirs(output_dir, exist_ok=True)
        
        save_path = os.path.join(output_dir, 'pred.json')
        save_json(pred_output, save_path)
    
    # Update Stats
    if part_query_count > 0:
        summary_stats[partition]['mrr'] = part_mrr_sum / part_query_count
        summary_stats[partition]['queries'] = part_query_count
    
    print(f"   ✅ Finished {partition}. MRR: {summary_stats[partition]['mrr']:.4f}")
    print(f"   ⏱️ Partition total: {time.time() - partition_start:.2f}s")
    print("-" * 50)

# In timing breakdown sau khi chạy xong
print_timing_stats()


🚀 STARTING NEW INFERENCE PIPELINE (refs.bib + references.json)...

🔵 Processing Partition: [TRAIN]
   ⏱️ Load labels: 0.14s
   ⏱️ Extract groundtruth: 0.01s
   ↳ Found 604 papers. Processing...


   Ranking train: 100%|██████████| 604/604 [1:08:24<00:00,  6.80s/it]


   ✅ Finished train. MRR: 0.9615
   ⏱️ Partition total: 4104.91s
--------------------------------------------------
🔵 Processing Partition: [VALIDATION]
   ⏱️ Load labels: 0.04s
   ⏱️ Extract groundtruth: 0.01s
   ↳ Found 2 papers. Processing...


   Ranking validation: 100%|██████████| 2/2 [00:41<00:00, 20.54s/it]


   ✅ Finished validation. MRR: 0.9657
   ⏱️ Partition total: 41.13s
--------------------------------------------------
🔵 Processing Partition: [TEST]
   ⏱️ Load labels: 0.02s
   ⏱️ Extract groundtruth: 0.00s
   ↳ Found 2 papers. Processing...


   Ranking test: 100%|██████████| 2/2 [00:19<00:00,  9.93s/it]

   ✅ Finished test. MRR: 0.9324
   ⏱️ Partition total: 19.90s
--------------------------------------------------

⏱️  TIMING BREAKDOWN
Phase                     | Total (s)    | Avg/Query (ms)  | %       
-----------------------------------------------------------------
Feature Extraction        |    3823.56s |       117.70ms |   98.8%
TF-IDF Cosine             |      15.27s |         0.47ms |    0.4%
Model Prediction          |      31.21s |         0.96ms |    0.8%
-----------------------------------------------------------------
TOTAL                     |    3870.04s |       119.13ms | 100.0%

📊 Processed 32485 queries


In [8]:
# --- FINAL REPORT ---
print("\n" + "="*40)
print("📊 SUBMISSION GENERATION REPORT")
print("="*40)
print(f"{'Partition':<15} | {'Papers':<10} | {'Queries':<10} | {'MRR':<10}")
print("-" * 50)

for part in PARTITIONS:
    stats = summary_stats[part]
    print(f"{part.upper():<15} | {stats['papers']:<10} | {stats['queries']:<10} | {stats['mrr']:.4f}")

print("="*40)
print(f"\n📁 All pred.json files are ready at: {os.path.abspath(SUBMISSION_DIR)}")


📊 SUBMISSION GENERATION REPORT
Partition       | Papers     | Queries    | MRR       
--------------------------------------------------
TRAIN           | 604        | 13339      | 0.9615
VALIDATION      | 2          | 107        | 0.9657
TEST            | 2          | 85         | 0.9324

📁 All pred.json files are ready at: d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\23127011\notebooks\submission_final\23127011


In [11]:
# Biến để thống kê tổng kết
PARTITIONS = ['validation', 'test']
summary_stats = {
    'validation': {'papers': 0, 'mrr': 0, 'queries': 0},
    'test': {'papers': 0, 'mrr': 0, 'queries': 0}
}

# Reset timing trước khi chạy
reset_timing_stats()

print("🚀 STARTING NEW INFERENCE PIPELINE (refs.bib + references.json)...\n")

for partition in PARTITIONS:
    print(f"🔵 Processing Partition: [{partition.upper()}]")
    partition_start = time.time()
    
    # 1. Load labels.json để lấy groundtruth và paper_ids
    t_load = time.time()
    label_file = os.path.join(DATASET_ROOT, partition, 'labels.json')
    if not os.path.exists(label_file):
        print(f"   ⚠️ Warning: File {label_file} not found. Skipping.")
        continue
        
    labels_data = load_json(label_file)
    if not labels_data:
        print("   ⚠️ Empty labels. Skipping.")
        continue
    print(f"   ⏱️ Load labels: {time.time() - t_load:.2f}s")

    # 2. Extract groundtruth và paper_ids
    t_transform = time.time()
    groundtruth_map = extract_groundtruth_from_labels(labels_data)
    paper_ids = list(groundtruth_map.keys())
    
    summary_stats[partition]['papers'] = len(paper_ids)
    print(f"   ⏱️ Extract groundtruth: {time.time() - t_transform:.2f}s")
    print(f"   ↳ Found {len(paper_ids)} papers. Processing...")
    
    # 3. NEW: Loop qua từng paper và load refs.bib + references.json
    part_mrr_sum = 0
    part_query_count = 0
    
    for paper_id in tqdm(paper_ids, desc=f"   Ranking {partition}"):
        # Xác định đường dẫn paper folder trong data_output_v2
        paper_dir = os.path.join(DATA_FOLDER, paper_id)
        
        if not os.path.exists(paper_dir):
            print(f"   ⚠️ Paper folder not found: {paper_id}")
            continue
        
        # 4. Load QUERIES từ refs.bib (TARGET entries)
        bib_path = os.path.join(paper_dir, 'refs.bib')
        queries = load_queries_from_bib(bib_path)
        
        if not queries:
            print(f"   ⚠️ No queries in {paper_id}/refs.bib")
            continue
        
        # 5. Load CANDIDATES từ references.json (Ground truth pool)
        ref_json_path = os.path.join(paper_dir, 'references.json')
        candidates = load_candidates_from_references(ref_json_path)
        
        if not candidates:
            print(f"   ⚠️ No candidates in {paper_id}/references.json")
            continue
        
        # 6. Run ranking
        predictions = rank_paper_fast(queries, candidates, model, feature_names)
        
        # 7. Build pred.json output
        groundtruth = groundtruth_map.get(paper_id, {})
        
        pred_output = {
            "partition": partition,
            "groundtruth": groundtruth,
            "prediction": predictions
        }
        
        # 8. Calculate MRR
        for query in queries:
            bib_key = query['key']
            true_id = groundtruth.get(bib_key)
            
            if not true_id:
                print("skip at partition ", partition, bib_key)
                continue
            
            top_5 = predictions.get(bib_key, [])
            
            if true_id in top_5:
                rank = top_5.index(true_id) + 1
                part_mrr_sum += 1.0 / rank
            
            part_query_count += 1
        
        # 9. Save pred.json
        safe_paper_id = str(paper_id).strip()
        output_dir = os.path.join(SUBMISSION_DIR, safe_paper_id)
        os.makedirs(output_dir, exist_ok=True)
        
        save_path = os.path.join(output_dir, 'pred.json')
        save_json(pred_output, save_path)
    
    # Update Stats
    if part_query_count > 0:
        summary_stats[partition]['mrr'] = part_mrr_sum / part_query_count
        summary_stats[partition]['queries'] = part_query_count
    
    print(f"   ✅ Finished {partition}. MRR: {summary_stats[partition]['mrr']:.4f}")
    print(f"   ⏱️ Partition total: {time.time() - partition_start:.2f}s")
    print("-" * 50)

# In timing breakdown sau khi chạy xong
print_timing_stats()


🚀 STARTING NEW INFERENCE PIPELINE (refs.bib + references.json)...

🔵 Processing Partition: [VALIDATION]
   ⏱️ Load labels: 0.00s
   ⏱️ Extract groundtruth: 0.00s
   ↳ Found 2 papers. Processing...


   Ranking validation:  50%|█████     | 1/2 [00:10<00:10, 10.19s/it]

skip at partition  validation ref_6
skip at partition  validation ref_8
skip at partition  validation ref_13
skip at partition  validation ref_14
skip at partition  validation ref_17
skip at partition  validation ref_22
skip at partition  validation ref_33
skip at partition  validation ref_35
skip at partition  validation ref_42
skip at partition  validation ref_48
skip at partition  validation ref_49
skip at partition  validation ref_50
skip at partition  validation ref_51
skip at partition  validation ref_52
skip at partition  validation ref_53
skip at partition  validation ref_60
skip at partition  validation ref_61
skip at partition  validation ref_62
skip at partition  validation ref_63
skip at partition  validation ref_75
skip at partition  validation ref_88


   Ranking validation: 100%|██████████| 2/2 [00:15<00:00,  7.69s/it]


skip at partition  validation ref_0
skip at partition  validation ref_1
skip at partition  validation ref_2
skip at partition  validation ref_3
skip at partition  validation ref_4
skip at partition  validation ref_5
skip at partition  validation ref_7
skip at partition  validation ref_8
skip at partition  validation ref_9
skip at partition  validation ref_11
skip at partition  validation ref_12
skip at partition  validation ref_13
skip at partition  validation ref_14
skip at partition  validation ref_15
skip at partition  validation ref_16
skip at partition  validation ref_17
skip at partition  validation ref_18
skip at partition  validation ref_21
skip at partition  validation ref_22
skip at partition  validation ref_23
skip at partition  validation ref_24
skip at partition  validation ref_31
skip at partition  validation ref_34
skip at partition  validation ref_36
skip at partition  validation ref_43
skip at partition  validation ref_46
skip at partition  validation ref_50
skip at pa

   Ranking test:  50%|█████     | 1/2 [00:01<00:01,  1.23s/it]

skip at partition  test ref_9
skip at partition  test ref_10
skip at partition  test ref_11
skip at partition  test ref_13
skip at partition  test ref_14
skip at partition  test ref_15
skip at partition  test ref_17
skip at partition  test ref_26
skip at partition  test ref_29
skip at partition  test ref_30
skip at partition  test ref_32


   Ranking test: 100%|██████████| 2/2 [00:06<00:00,  3.20s/it]

skip at partition  test ref_27
skip at partition  test ref_36
skip at partition  test ref_45
skip at partition  test ref_65
skip at partition  test ref_67
   ✅ Finished test. MRR: 0.9324
   ⏱️ Partition total: 6.41s
--------------------------------------------------

⏱️  TIMING BREAKDOWN
Phase                     | Total (s)    | Avg/Query (ms)  | %       
-----------------------------------------------------------------
Feature Extraction        |      20.67s |        76.86ms |   99.1%
TF-IDF Cosine             |       0.06s |         0.24ms |    0.3%
Model Prediction          |       0.13s |         0.48ms |    0.6%
-----------------------------------------------------------------
TOTAL                     |      20.87s |        77.58ms | 100.0%

📊 Processed 269 queries
